# 1. The Robot Simulator

In [ ]:
import jyrobot

In [ ]:
world = jyrobot.World(width=250, height=100)

In [ ]:
world.watch()

In [ ]:
robot = jyrobot.Scribbler()

In [ ]:
world.add_robot(robot)
world.update()

In [ ]:
robot.set_pose(200, 50, 180)
world.update()

In [ ]:
robot.add_device(jyrobot.RangeSensor(width=0, max=100))
world.update()

In [ ]:
world.save_as("experiment-01")

In [ ]:
world.reset()

# 2. Data Collection

In [ ]:
world.update()

In [ ]:
data = []

def controller(robot):
    reading = robot["laser"].get_reading()
    speed = round((reading - 0.5)/0.5, 1)
    robot.forward(speed)
    data.append([reading, speed, world.time])
    if speed == 0:
        return True

In [ ]:
world.run([controller], show=True, real_time=True)

# 3. Analysis: Dataset

In [ ]:
import pandas #, pandas_profiling
import tensorflow as tf

In [ ]:
df = pandas.DataFrame(data, columns=["range", "translation", "time"])

In [ ]:
df

In [ ]:
# pandas_profiling.ProfileReport(df)

# 4. Training

## 4.1 Training: Dataset

In [ ]:
targets = df['translation']

In [ ]:
targets.values.shape

In [ ]:
inputs = df['range']

In [ ]:
inputs.values.shape

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
    ([[v] for v in inputs.values], 
     [[v/2 + 0.5] for v in targets.values]))

In [ ]:
#for i in dataset:
#    print(i[0].numpy(), i[1].numpy())

## 4.2 Training: Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [ ]:
input = Input((1,))

In [ ]:
hidden = Dense(10, activation="sigmoid")(input)

In [ ]:
output = Dense(1, activation="sigmoid")(hidden)

In [ ]:
model = Model(input, output)

In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

In [ ]:
model.predict(dataset).shape

# 5. Analysis: Training

In [ ]:
model.fit(dataset, epochs=10, verbose=0)

In [ ]:
model.predict([0.5])

In [ ]:
#model.predict(dataset)

In [ ]:
world.reset()

In [ ]:
def controller(robot):
    reading = robot["laser"].get_reading()
    outputs = model.predict([[reading]])
    speed = round((outputs[0][0] - 0.5) * 2.0, 1)
    robot.forward(speed)
    if speed == 0:
        return True

In [ ]:
plot = world[0].plot(lambda robot: (robot.world.time, robot["laser"].get_reading()), 
           y_label="reading",
           x_label="time",
           title="robot['laser'].get_reading()")
plot.watch()

In [ ]:
world.run([controller], show=True, real_time=True)